In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
import os
import librosa 
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Model
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam, Nadam
from keras import regularizers
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

In [2]:
normal = sorted(glob(os.path.join("", "normal", "*.wav")))
abnormal = sorted(glob(os.path.join("", "abnormal", "*.wav")))

In [3]:
x=abnormal+normal

In [4]:
y_t= [[0]]*408
n = [[1]]*533
y_t = y_t + n
y = np.array(y_t)

In [6]:
rfy_t= [0]*408
rfn = [1]*533
rfy_t = rfy_t + rfn
rfy = np.array(rfy_t)

In [7]:
x_train, x_test = train_test_split(x, test_size=0.3, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42)
rfy_train, rfy_test = train_test_split(rfy, test_size=0.3, random_state=42)

In [8]:
def load_data(path):
    x_t = []
    for i in path:
        data, sr = librosa.load(i, sr=8000)
        data=data[:147000]
        new_sample_rate = 1000
        resampled_data = librosa.resample(data, orig_sr=sr, target_sr=new_sample_rate)
        x_t.append(resampled_data)
    return x_t

In [9]:
X_train=load_data(x_train)
X_test=load_data(x_test)

In [10]:
def wavelet_transform(data):
    for i in range(len(data)):
        coeffs = pywt.wavedec(data[i], wavelet='haar', level=5)
        new = np.zeros(0)
        for j in coeffs:
            new = np.concatenate((new, j), axis=-1)
        data[i] = new

In [11]:
wavelet_transform(X_train)
wavelet_transform(X_test)

In [12]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [34]:
model4 = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(128, 16, input_shape=(18378,1), kernel_regularizer=regularizers.l2(l=0.01)),
    tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Conv1D(32, 16,kernel_regularizer=regularizers.l2(l=0.01)),
    tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(8, 4, kernel_regularizer=regularizers.l2(l=0.01)),
    tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

opt4 = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn4 = tf.keras.losses.BinaryCrossentropy()
model4.compile(optimizer = opt4,
              loss = loss_fn4,
              metrics = ['accuracy'])
model4.fit(X_train, y_train, epochs=15)

_, test_accuracy = model4.evaluate(X_test, y_test, verbose=0)
print(test_accuracy)


Epoch 1/15
21/21 [==============================] - 51s 2s/step - loss: 1.2326 - accuracy: 0.5471
Epoch 2/15
21/21 [==============================] - 46s 2s/step - loss: 1.0140 - accuracy: 0.5942
Epoch 3/15
21/21 [==============================] - 46s 2s/step - loss: 0.8922 - accuracy: 0.6018
Epoch 4/15
21/21 [==============================] - 45s 2s/step - loss: 0.7928 - accuracy: 0.6489
Epoch 5/15
21/21 [==============================] - 46s 2s/step - loss: 0.7136 - accuracy: 0.7158
Epoch 6/15
21/21 [==============================] - 47s 2s/step - loss: 0.6267 - accuracy: 0.7371
Epoch 7/15
21/21 [==============================] - 46s 2s/step - loss: 0.5764 - accuracy: 0.7675
Epoch 8/15
21/21 [==============================] - 50s 2s/step - loss: 0.5472 - accuracy: 0.7842
Epoch 9/15
21/21 [==============================] - 47s 2s/step - loss: 0.5212 - accuracy: 0.8040
Epoch 10/15
21/21 [==============================] - 46s 2s/step - loss: 0.5223 - accuracy: 0.7766
Epoch 11/15
21/21 [

In [40]:
prediction = model4.predict(X_test[22].reshape(1,18378))  # a zero
print(f" predicting a one: {prediction}")
print(f"y_hat : {y_test[22]}")
prediction = model4.predict(X_test[20].reshape(1,18378))  # a one
print(f" predicting a zero:  {prediction}")
print(f"y_hat : {y_test[20]}")

1/1 [==============================] - 0s 51ms/step
 predicting a one: [[0.83666426]]
y_hat : [1]
1/1 [==============================] - 0s 24ms/step
 predicting a zero:  [[0.40125242]]
y_hat : [0]
